In [43]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn. preprocessing import LabelEncoder
import pickle


In [44]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='T6g4d-hWoe54bRjs4L8eItmWdWrgGl1O3wlus9a69K3S',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'mlrcarresalepricepredict-donotdelete-pr-wipybxw1qxkwb2'
object_key = 'autos.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xdc in position 14: invalid continuation byte

In [ ]:

df[df.seller != 'gewerblich']
df=df.drop( 'seller', 1)
df[df.offerType != 'Gesuch']
df=df.drop( 'offerType', 1)

In [ ]:
df = df[ (df.powerPS > 50) & (df.powerPS < 900) ]
df = df[ (df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]

In [ ]:
df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'],axis='columns',inplace=True)

In [ ]:
new_df = df.copy()
new_df = new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])

In [ ]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'),inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electirc'),inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [ ]:
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]

In [ ]:
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df['gearbox'].fillna(value='not-declared',inplace=True)
new_df['vehicleType'].fillna(value='not-declared',inplace=True)
new_df['model'].fillna(value='not-declared',inplace=True)

In [ ]:
new_df.to_csv("autos_preprocessed.csv")

#label encoding the categorical data
labels = ['gearbox','notRepairedDamage','model','brand','fuelType','vehicleType']

In [11]:
mapper = {}
for i in labels:
	mapper[i] = LabelEncoder()
	mapper[i].fit(new_df[i])
	tr = mapper[i].transform(new_df[i])
	np.save(str('classes'+i+'.npy'),mapper[i].classes_)
	print(i,";",mapper[i])
	new_df.loc[:,i+'_labels'] = pd.Series(tr,index = new_df.index)
labeled = new_df[ [ 'price' , 'yearOfRegistration','powerPS','kilometer','monthOfRegistration'] + [x+"_labels" for x in labels]]

gearbox ; LabelEncoder()
notRepairedDamage ; LabelEncoder()
model ; LabelEncoder()
brand ; LabelEncoder()
fuelType ; LabelEncoder()
vehicleType ; LabelEncoder()


In [ ]:
print(labeled.columns)

In [ ]:
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values

In [ ]:
Y = Y.reshape(-1,1)
from sklearn.model_selection import cross_val_score , train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=3)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators = 1000,max_depth=10,random_state=34)

In [ ]:
regressor.fit(X_train,np.ravel(Y_train,order='C'))

In [ ]:
y_pred = regressor.predict(X_test)
print(r2_score(Y_test,y_pred))

# Deployment

In [26]:
!pip install -U ibm-watson-machine-learning

In [27]:
from ibm_watson_machine_learning import APIClient

In [28]:
wml_credentials={
    "apikey":"rEO0JnljvPMXhdlXBZGiyM4xUnCzL3lM3Pjs9pRXNkKy",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [29]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------  ------------------------
ID                                    NAME       CREATED
5a078b6d-4677-436b-a60a-3628a133e4a5  Car_space  2022-11-16T07:01:54.376Z
------------------------------------  ---------  ------------------------


In [30]:
space_id="5a078b6d-4677-436b-a60a-3628a133e4a5"

In [31]:
wml_client.set.default_space(space_id)

'SUCCESS'

In [32]:
wml_client.software_specifications.list(500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [41]:
MODEL_NAME = 'Car_space'
DEPLOYMENT_NAME='car_deploy'
DEMO_MODEL = y_predict


NameError: name 'y_predict' is not defined

In [34]:
#set python version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [36]:
#setup model data
model_props={
    wml_client.repository.ModelMetaNames.NAME:MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE:'scikit-learn_1.0',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

In [39]:
#save model

model_details = wml_client.repository.store_model(
    model=CAR_SPACE,
    meta_props = model_props,
    training_data = x_train ,
    training_target = y_train

)

NameError: name 'CAR_SPACE' is not defined

In [40]:
model_details

NameError: name 'model_details' is not defined